<a href="https://colab.research.google.com/github/rafaelhab/unsupervisedlearning/blob/master/CLUSTERING_7LABELED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

De los datasets de UCI Machine Learning, tomé Activity Recognition from Single Chest-Mounted Accelerometer Data Set <br>

La URL es: https://archive.ics.uci.edu/ml/machine-learning-databases/00287/Activity%20Recognition%20from%20Single%20Chest-Mounted%20Accelerometer.zip

https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+from+Single+Chest-Mounted+Accelerometer


Source:<br>

Uncalibrated Accelerometer Data are collected from 15 participantes performing 7 activities. The dataset provides challenges for identification and authentication of people using motion patterns.


Data Set Information:<br>

--- The dataset collects data from a wearable accelerometer mounted on the chest <br>
--- Sampling frequency of the accelerometer: 52 Hz <br>
--- Accelerometer Data are Uncalibrated <br>
--- Number of Participants: 15 <br>
--- Number of Activities: 7 <br>
--- Data Format: CSV<br>

Attribute Information:<br>

--- Data are separated by participant <br>
--- Each file contains the following information <br>
---- sequential number, x acceleration, y acceleration, z acceleration, label <br>
--- Labels are codified by numbers <br>
--- 1: Working at Computer <br>
--- 2: Standing Up, Walking and Going updown stairs <br>
--- 3: Standing <br>
--- 4: Walking <br>
--- 5: Going UpDown Stairs <br>
--- 6: Walking and Talking with Someone <br>
--- 7: Talking while Standing <br>

In [0]:
#Importamos las librerías que vamos a usar:
import requests
import io
import zipfile
import pandas as pd
from sklearn.cluster import KMeans

In [0]:
#Función para extraer el archivo ZIP
def download_extract_zip(url):
    """
    Download a ZIP file and extract its contents in memory
    yields (filename, file-like object) pairs
    """
    response = requests.get(url)
    with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
        df = pd.DataFrame()
        for i in range(1,10):
            temp_df = pd.read_csv(thezip.open('Activity Recognition from Single Chest-Mounted Accelerometer/'+str(i)+'.csv'),header=None)
            df = df.append(temp_df, ignore_index = True)
    
    return df

In [0]:
#Extracción del archivo
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00287/Activity%20Recognition%20from%20Single%20Chest-Mounted%20Accelerometer.zip'

Dataset=download_extract_zip(url)

Dataset.head()

,0,1,2,3,4
0,0.0,1502,2215,2153,1
1,1.0,1667,2072,2047,1
2,2.0,1611,1957,1906,1
3,3.0,1601,1939,1831,1
4,4.0,1643,1965,1879,1


In [0]:
len(Dataset)

1293689

In [0]:
Dataset=Dataset.dropna() #Borrar NaN
Dataset=Dataset[(Dataset[[4]] != 0).all(axis=1)] #Borrar labels con 0 ya que según la descripción no existen

In [0]:
#Tomando en cuenta que la columna 0 es un número secuencial y la columna 4 el label, 
#tomaremos de las columnas 1 a 3 para aplicar el Kmeans
X = Dataset.iloc[: , [1,3]].values

#los labels son 7 por esto el Kmeans lo dejaré en 7 sin aplicar el elbow method.

kmeans = KMeans(n_clusters = 7, init='k-means++', max_iter=300, n_init=10, random_state=0)
y_kmeans = kmeans.fit_predict(X)

In [0]:
#Agrego al final del Dataset la columna means para analizar los resultados
Dataset['kmeans']=y_kmeans+1

In [0]:
#Revisión de los labels
Dataset[4].value_counts()

7    489031
1    338942
4    222560
3    143721
6     34371
2     30938
5     30413
Name: 4, dtype: int64

In [0]:
#Revisión de los predecidos
Dataset['kmeans'].value_counts()

2    293951
1    271772
6    268459
4    236318
3     89880
7     80791
5     48805
Name: kmeans, dtype: int64

In [0]:
#R cuadrado
from sklearn.metrics import r2_score
r2_score(Dataset[4], Dataset['kmeans'])

-0.7386716193159357

In [0]:
#RMSSTD homogeneidad de los clústeres del agrupamiento
from sklearn.metrics.cluster import homogeneity_score
homogeneity_score(Dataset.iloc[:100000,4], Dataset.iloc[:100000,5])
#Corté el arreglo porque me estaba dando un problema que aún no resuelvo de overflow

0.3049028144086031

In [0]:
from sklearn.metrics import mutual_info_score, silhouette_score, calinski_harabaz_score
#davies_bouldin_score me da error
print(mutual_info_score(Dataset.iloc[:100000,4], Dataset.iloc[:100000,5]))

0.4726378064034043


In [0]:
print(calinski_harabaz_score(X, Dataset['kmeans']))

984278.6998985157
